## Load the data

In [1]:
import pandas as pd

In [2]:
url = "https://docs.google.com/spreadsheets/d/e/2PACX-1vScPt_dOJIulsY96YvXYVVR4PZWSpJfMJKjjJowaz_P_bwRfkAHxVViNG8_mm7Dpc_44bvLO0cwkfLD/pub?gid=212407814&single=true&output=csv"
messy = pd.read_csv(url)
messy

,First Name,Last Name,Date of Birth,Library Card Expired?,Book,Author_A,Author_B,Genres,Days Late,Late Fees
0,Danny,Torrance,1971-12-11,Yes,The Phantom Tollbooth,Norton Juster,Jules Feiffer,Fiction; Youth,15,$0.45
1,Danny,Torrance,1971-12-11,Yes,A Wrinkle in Time,Madeleine L'Engle,-,Fiction; Youth,15,$0.45
2,Jack,Torrance,1946-08-03,No,Pride and Prejudice,Jane Austen,-,Fiction,7,$0.70
3,Jack,Torrance,1946-08-03,No,I Know Why the Caged Bird Sings,Maya Angelou,-,Nonfiction; Memoir,15,$1.50
4,Jack,Torrance,1946-08-03,No,A Wrinkle in Time,Madeleine L'Engle,-,Fiction; Youth,15,$0.45
5,Wendy,Torrance,1946-03-05,No,Emma,Jane Austen,-,Fiction,7,$0.70
6,Wendy,Torrance,1946-03-05,No,The Electric Kool-Aid Acid Test,Tom Wolfe,-,Nonfiction,15,$4.50
7,Wendy,Torrance,1946-03-05,No,The Communist Manifesto,Karl Marx,Friedrich Engels,Nonfiction,15,$6.00
8,Wendy,Torrance,1946-03-05,No,A Wrinkle in Time,Madeleine L'Engle,-,Fiction; Youth,15,$0.45
9,?,Massey,?,No,Pride and Prejudice,Jane Austen,-,Fiction,15,$1.50


## Make tidy tables

In [3]:
tidy_person = (
    messy
    [[
        "First Name",
        "Last Name",
        "Date of Birth",
        "Library Card Expired?"
    ]]
    .drop_duplicates()
    .reset_index(drop=True)
    .reset_index(names="person_id")
)

tidy_person

,person_id,First Name,Last Name,Date of Birth,Library Card Expired?
0,0,Danny,Torrance,1971-12-11,Yes
1,1,Jack,Torrance,1946-08-03,No
2,2,Wendy,Torrance,1946-03-05,No
3,3,?,Massey,?,No


In [4]:
tidy_book = (
    messy
    [[
        "Book",
        "Author_A",
        "Author_B",
        "Genres"
    ]]
    .drop_duplicates()
    .reset_index(drop=True)
    .reset_index(names="book_id")
)

tidy_book

,book_id,Book,Author_A,Author_B,Genres
0,0,The Phantom Tollbooth,Norton Juster,Jules Feiffer,Fiction; Youth
1,1,A Wrinkle in Time,Madeleine L'Engle,-,Fiction; Youth
2,2,Pride and Prejudice,Jane Austen,-,Fiction
3,3,I Know Why the Caged Bird Sings,Maya Angelou,-,Nonfiction; Memoir
4,4,Emma,Jane Austen,-,Fiction
5,5,The Electric Kool-Aid Acid Test,Tom Wolfe,-,Nonfiction
6,6,The Communist Manifesto,Karl Marx,Friedrich Engels,Nonfiction
7,7,Slouching Towards Bethlehem,Joan Didion,-,Nonfiction


In [5]:
tidy_book_author = (
    tidy_book
    .melt(
        id_vars = ["book_id"],
        value_vars = [ "Author_A", "Author_B" ],
        value_name = "Author",
    )
    .assign(
        is_first_author = lambda df: df["variable"].str.contains("_A")
    )
    .loc[lambda df: df["Author"] != "-"]
    .drop(columns = [ "variable" ])
)

tidy_book_author

,book_id,Author,is_first_author
0,0,Norton Juster,True
1,1,Madeleine L'Engle,True
2,2,Jane Austen,True
3,3,Maya Angelou,True
4,4,Jane Austen,True
5,5,Tom Wolfe,True
6,6,Karl Marx,True
7,7,Joan Didion,True
8,0,Jules Feiffer,False
14,6,Friedrich Engels,False


In [6]:
tidy_book_genre = (
    tidy_book
    .assign(
        genre = lambda df: df["Genres"].str.split("; ")
    )
    [[
        "book_id",
        "genre",
    ]]
    .explode("genre")
    .drop_duplicates()
)

tidy_book_genre

,book_id,genre
0,0,Fiction
0,0,Youth
1,1,Fiction
1,1,Youth
2,2,Fiction
3,3,Nonfiction
3,3,Memoir
4,4,Fiction
5,5,Nonfiction
6,6,Nonfiction


In [7]:
tidy_borrowings = (
    messy
    .merge(tidy_person)
    .merge(tidy_book)
    .assign(**{
        "Late Fees": lambda df: df["Late Fees"].str.strip("$").astype(float)
    })
    [[
        "person_id",
        "book_id",
        "Days Late",
        "Late Fees"
    ]]
)

tidy_borrowings

,person_id,book_id,Days Late,Late Fees
0,0,0,15,0.45
1,0,1,15,0.45
2,1,2,7,0.70
3,1,3,15,1.50
4,1,1,15,0.45
5,2,4,7,0.70
6,2,5,15,4.50
7,2,6,15,6.00
8,2,1,15,0.45
9,3,2,15,1.50


In [8]:
assert len(tidy_borrowings) == len(messy)

## Just for fun ...

Which has accumulated more late fees, fiction or nonfiction?

In [9]:
(
    tidy_book_genre
    .merge(
        tidy_borrowings
    )
    .groupby("genre")
    ["Late Fees"]
    .sum()
    .sort_values(ascending=False)
)

genre
Nonfiction    13.5
Fiction        4.7
Youth          1.8
Memoir         1.5
Name: Late Fees, dtype: float64

How many fines have been accumulated for books by authors with an "F"/"f" in their name?

In [10]:
(
    tidy_book_author
    .loc[lambda df: df["Author"].str.contains("f", case=False)]
    .merge(tidy_borrowings)
    ["Late Fees"]
    .sum()
    .item()
)

10.95

What borrower-genre combination is responsible for the greatest late fees? (OK to double-count when a book has multiple genres.)

In [11]:
(
    tidy_borrowings
    .merge(tidy_book_genre)
    .groupby([ "person_id", "genre" ])
    [["Late Fees"]]
    .sum()
    .reset_index()
    .merge(tidy_person)
    [[
        "First Name",
        "Last Name",
        "genre",
        "Late Fees"
    ]]
    .sort_values("Late Fees", ascending=False)
)

,First Name,Last Name,genre,Late Fees
7,Wendy,Torrance,Nonfiction,10.50
3,Jack,Torrance,Memoir,1.50
4,Jack,Torrance,Nonfiction,1.50
9,?,Massey,Fiction,1.50
10,?,Massey,Nonfiction,1.50
2,Jack,Torrance,Fiction,1.15
6,Wendy,Torrance,Fiction,1.15
0,Danny,Torrance,Fiction,0.90
1,Danny,Torrance,Youth,0.90
5,Jack,Torrance,Youth,0.45


---

---

---